In [1]:
import numpy as np
import pandas as pd
import math
import random
import datetime
import itertools

In [2]:
# function for probabilities
def compute_probabilities(s1, s2):
    

    p12 = 1/60 + (1/200 - 1/60) * math.sqrt(1-((10-s1)/10)**2)
    p23 = 1/30 + (1/100 - 1/30) * math.sqrt(1-((10-s2)/10)**3)
    
    return p12, p23

In [3]:
# function for 60days scheduling
def scheduling(p12, p23, s1, s2):
    
    #for the 1st day
    
    n12 = 2000
    B12 = np.random.binomial(n12, p12)

        
    W1 = B12 
    B1 = 0
    
    N1 = 2000 - W1 - B1
    
    total_gain = 2000 - (s1+s2)
    last_gain = 0
    
    #all the other
    for i in range(1, 60):
        
        W0 = W1
        B0 = B1
        n12 = 2000 - W0- B0
        B12 = np.random.binomial(n12, p12)
        
        n23 = W0
        B23 = np.random.binomial(n23, p23)
        
        W1 = W0 + B12 - B23
        B1 = B23
        
        N1 = 2000 - W1 - B1
        
        daily_gain = (2000 - B1) * 1 - 90 * B1 -(s1+s2)
        
        total_gain = total_gain +  daily_gain
        
        if i== 59:
            last_gain = (2000 - B1) * 1 - 90 * B1
            
    
    
    return total_gain, last_gain

## Simulation

In [4]:
now = datetime.datetime.now()

s_list = np.arange(2,11, 2)

combinations = list(itertools.product(s_list, s_list))

result = []

tot = []

# Start the simulation
for s1, s2 in combinations:
        
    p12, p23 = compute_probabilities(s1, s2)
    
    result.append(s1)
    result.append(s2)
    tot.append(s1)
    tot.append(s2)
    
    i_result = []
    tot_g = []
    for i in range(10000):

        tot_gain, last_gain = scheduling(p12, p23, s1, s2)
        i_result.append(last_gain)
        tot_g.append(tot_gain)
    result.append(i_result)
    tot.append(tot_g)

finish = datetime.datetime.now()

delta = finish - now

print(f"Start: {now}\nFinish: {finish}\nDeltaT:{delta}" )

Start: 2021-02-11 10:59:49.156687
Finish: 2021-02-11 11:01:09.941142
DeltaT:0:01:20.784455


## Expected value for each pair (s1, s2)

In [5]:
expected_values = {}
array = []

final = []
for r, (s1, s2) in zip(tot[2::3], combinations):
    expected_values[(s1,s2)] = sum(r)/10000
    array.append(sum(r)/10000)
 

In [6]:
values = np.array(array)
values
from scipy.stats import zscore
from sklearn import preprocessing

scaler = preprocessing.StandardScaler()
mean_training = values.mean()
std_training = values.std()

max_training = max(values) # max values for normalization
min_training = min(values) # min values for norm.

# Max-min norm.
normalized = [(x - min_training)/(max_training - min_training) for x in values]
#standardized = [(x - mean_training)/(std_training) for x in values]

final = []
# Max-min Normalization for the output of the NN
for (s1, s2) , st_value in zip(combinations, normalized):
    final.append([[s1,s2], [st_value]])
    print(f"Pair:{s1,s2} --> {st_value} ",)

Pair:(2, 2) --> 0.0 
Pair:(2, 4) --> 0.3311354322174388 
Pair:(2, 6) --> 0.4886641872101385 
Pair:(2, 8) --> 0.5428516942242185 
Pair:(2, 10) --> 0.5430060508014837 
Pair:(4, 2) --> 0.34609899665317106 
Pair:(4, 4) --> 0.6037512457784676 
Pair:(4, 6) --> 0.7260188002279828 
Pair:(4, 8) --> 0.7683533129353912 
Pair:(4, 10) --> 0.7675838265965866 
Pair:(6, 2) --> 0.5588982237358103 
Pair:(6, 4) --> 0.771452587598947 
Pair:(6, 6) --> 0.8705406542037925 
Pair:(6, 8) --> 0.9036107266392673 
Pair:(6, 10) --> 0.904583536285382 
Pair:(8, 2) --> 0.6767335745253623 
Pair:(8, 4) --> 0.861566820829729 
Pair:(8, 6) --> 0.951930850421799 
Pair:(8, 8) --> 0.9804628808826533 
Pair:(8, 10) --> 0.9777797839987649 
Pair:(10, 2) --> 0.7089735337386778 
Pair:(10, 4) --> 0.8877219738851595 
Pair:(10, 6) --> 0.9728775026873053 
Pair:(10, 8) --> 1.0 
Pair:(10, 10) --> 0.9994269423977189 


In [7]:
mean_training

100983.62918000002

## Neaural Network

In [8]:
# Normalization of the input
X_train = np.zeros((25,2))
for i, (s1, s2) in enumerate(combinations):
    X_train[i,0] = s1/10
    X_train[i,1] = s2/10

# output normalized
y_train = normalized


In [9]:
# activation function
def sigmoid(x): 
    return 1/(1+ np.exp(-x))

def sigmoid_derivative(x):
    
    return x*(1-x)

In [10]:
# function: convert normalized output to denormalized gain
def convert_to_gain(x):
    return x*(max_training-min_training) +  min_training


In [11]:
import numpy as np

class NeuralNet():
    def __init__(self):
        np.random.seed(41)
        self.bias = np.random.rand(1) # o lo metto fisso ad 1?
        self.learning_rate = 0.2
    
    def update_weights(self, weights):
        self.weights = weights

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def sigmoid_der(self, x):
        return self.sigmoid(x)*(1-self.sigmoid(x))

    def training(self, X_train, output, tol):
       
        # 
        
        bias = [1, 1, 1]  # 2 for the 2 hidden neurons and 1 for the output
        
        weights = 0.5*np.random.rand(3, 3) # weights initialization 
        #print(weights)
        
        # step lenght coefficient, mu
        coeff = 0.01
        
        #Initialization: SSE initialized at high value, flag: boolean for early stop (reach tollerance), n_epochs
        sse_old = 10000
        flag = False
        n_epocs = 10000
        
        
        for i in range(n_epocs):
            
            if flag == True:
                print("Reached tol: ", tol)
                break
            
            out = np.zeros(25)
            
            numInput = X_train.shape[0]#len(X_train) #25 length of input

            for j in range(numInput):
                
                x2= [0,0]
                
                # Feed forward
                
                # Hidden Layer
                
                # Hidden node 1
                H1 = bias[0]*weights[0,0]+ X_train[j,0]*weights[0,1] + X_train[j,1]*weights[0,2];
                
                
                x2[0] = sigmoid(H1);
    
                # Hidden node 2
                H2 = bias[1]*weights[1,0]+ X_train[j,0]*weights[1,1] + X_train[j,1]*weights[1,2];
        
                x2[1] = sigmoid(H2);
                
                # Output layer
                x3_1 = bias[2]*weights[2,0] + x2[0]*weights[2,1] + x2[1]*weights[2,2];
                out[j] = sigmoid(x3_1);

                # Backpropagation 
                
                delta3_1 = out[j]*(1-out[j])*(output[j]-out[j]);
                
                # Propagate the delta backwards into hidden layers
                delta2_1 = x2[0]*(1-x2[0])*weights[2,1]*delta3_1;
                delta2_2 = x2[1]*(1-x2[1])*weights[2,2]*delta3_1;
                

                # Add weight changes to original weights
                # And use the new weights to repeat process.
                # delta weight = coeff*x*delta
                for k in range(3):# 1:3
                    if k == 0: # Bias cases
                        weights[0,k] = weights[0,k] + 2*coeff*bias[0]*delta2_1;
                        weights[1,k] = weights[1,k] + 2*coeff*bias[1]*delta2_2;
                        weights[2,k] = weights[2,k] + 2*coeff*bias[2]*delta3_1;
                    else: #% When k=2 or 3 input cases to neurons
                        weights[0,k] = weights[0,k] + 2*coeff*X_train[j,0]*delta2_1;
                        weights[1,k] = weights[1,k] + 2*coeff*X_train[j,0]*delta2_2;
                        weights[2,k] = weights[2,k] + 2*coeff*x2[k-1]*delta3_1;
                    
                # methods: update weights
                self.update_weights(weights)
            
            # compute the SSE
            sse = (output-out)**2
            print("Error: ",sse.sum())
            
            # Early stop, reached tollerance
            if abs(sse.sum() -sse_old)< tol:
                print(f"Reached at epoch {i}: ",abs(sse.sum() -sse_old))
                flag = True
                break
            # update sse_old
            sse_old = sse.sum()
            # update coeff
            coeff = coeff*0.9999
        return out
    
    # methods for prediction
    def think(self, x_valid):
        
        bias = [1,1,1]
        
        weights = self.weights
        
        x2= [0,0]
                # Hidden node 1
        H1 = bias[0]*weights[0,0]+ x_valid[0]*weights[0,1] + x_valid[1]*weights[0,2];
                
                
        x2[0] = sigmoid(H1);
    
                # Hidden node 2
        H2 = bias[1]*weights[1,0]+ x_valid[0]*weights[1,1] + x_valid[1]*weights[1,2];
        
        x2[1] = sigmoid(H2);
                
                # Output layer
        x3_1 = bias[2]*weights[2,0] + x2[0]*weights[2,1] + x2[1]*weights[2,2];
        out = sigmoid(x3_1);
        
        return out


In [12]:
## Training NN
net = NeuralNet()

tol = 10**(-6)
out= net.training(X_train, y_train, tol)


Error:  1.5617582300217967
Error:  1.5551589079416939
Error:  1.5491159229597398
Error:  1.5435779182919929
Error:  1.5384986762270032
Error:  1.5338365622965724
Error:  1.5295540335980622
Error:  1.52561720353131
Error:  1.5219954561578197
Error:  1.5186611042241691
Error:  1.515589085626227
Error:  1.5127566937360715
Error:  1.5101433375792672
Error:  1.507730328345605
Error:  1.5055006891497464
Error:  1.5034389853370853
Error:  1.5015311729612084
Error:  1.4997644633486054
Error:  1.4981272019190432
Error:  1.4966087596509197
Error:  1.495199435774028
Error:  1.493890370441081
Error:  1.4926734662771395
Error:  1.4915413178355261
Error:  1.4904871481022177
Error:  1.4895047512901367
Error:  1.4885884412520567
Error:  1.48773300491746
Error:  1.4869336602260959
Error:  1.486186018090233
Error:  1.4854860479698389
Error:  1.4848300466909117
Error:  1.4842146101778064
Error:  1.483636607806236
Error:  1.4830931591153347
Error:  1.4825816126452038
Error:  1.4820995266912023
Error:  1.4

Error:  1.4362377030178446
Error:  1.4360655745215483
Error:  1.4358932422064914
Error:  1.4357207058965449
Error:  1.4355479654156174
Error:  1.435375020587654
Error:  1.4352018712366394
Error:  1.435028517186598
Error:  1.4348549582615968
Error:  1.4346811942857445
Error:  1.4345072250831954
Error:  1.4343330504781473
Error:  1.4341586702948448
Error:  1.433984084357578
Error:  1.4338092924906864
Error:  1.4336342945185572
Error:  1.433459090265625
Error:  1.433283679556375
Error:  1.4331080622153418
Error:  1.4329322380671103
Error:  1.432756206936315
Error:  1.4325799686476415
Error:  1.4324035230258272
Error:  1.432226869895659
Error:  1.4320500090819752
Error:  1.431872940409665
Error:  1.4316956637036704
Error:  1.4315181787889804
Error:  1.431340485490638
Error:  1.4311625836337354
Error:  1.4309844730434151
Error:  1.430806153544869
Error:  1.43062762496334
Error:  1.4304488871241186
Error:  1.4302699398525445
Error:  1.4300907829740062
Error:  1.4299114163139393
Error:  1.429

Error:  1.3628046416673105
Error:  1.3625508260082317
Error:  1.3622967423603018
Error:  1.3620423905121124
Error:  1.3617877702520431
Error:  1.3615328813682583
Error:  1.3612777236487095
Error:  1.361022296881135
Error:  1.3607666008530594
Error:  1.3605106353517937
Error:  1.3602544001644372
Error:  1.3599978950778748
Error:  1.3597411198787803
Error:  1.3594840743536145
Error:  1.3592267582886244
Error:  1.3589691714698475
Error:  1.3587113136831082
Error:  1.3584531847140195
Error:  1.3581947843479822
Error:  1.357936112370187
Error:  1.3576771685656135
Error:  1.35741795271903
Error:  1.3571584646149955
Error:  1.356898704037858
Error:  1.3566386707717557
Error:  1.3563783646006184
Error:  1.356117785308165
Error:  1.3558569326779069
Error:  1.355595806493145
Error:  1.355334406536975
Error:  1.3550727325922816
Error:  1.3548107844417439
Error:  1.3545485618678315
Error:  1.354286064652811
Error:  1.3540232925787377
Error:  1.3537602454274642
Error:  1.3534969229806362
Error:  1.

Error:  1.2243943228198506
Error:  1.224002800074102
Error:  1.2236109131352617
Error:  1.2232186618437735
Error:  1.2228260460408735
Error:  1.2224330655685993
Error:  1.2220397202697897
Error:  1.2216460099880952
Error:  1.2212519345679769
Error:  1.2208574938547148
Error:  1.2204626876944096
Error:  1.2200675159339933
Error:  1.2196719784212262
Error:  1.2192760750047054
Error:  1.2188798055338723
Error:  1.218483169859011
Error:  1.2180861678312593
Error:  1.2176887993026098
Error:  1.2172910641259127
Error:  1.216892962154888
Error:  1.2164944932441237
Error:  1.2160956572490815
Error:  1.2156964540261033
Error:  1.2152968834324167
Error:  1.214896945326136
Error:  1.214496639566272
Error:  1.2140959660127326
Error:  1.2136949245263302
Error:  1.2132935149687851
Error:  1.2128917372027315
Error:  1.2124895910917215
Error:  1.21208707650023
Error:  1.2116841932936608
Error:  1.2112809413383494
Error:  1.2108773205015688
Error:  1.2104733306515363
Error:  1.2100689716574153
Error:  

Error:  1.0462887920472794
Error:  1.0457572443038925
Error:  1.0452253763277233
Error:  1.0446931886194464
Error:  1.044160681682142
Error:  1.0436278560213001
Error:  1.0430947121448162
Error:  1.0425612505629962
Error:  1.042027471788555
Error:  1.0414933763366174
Error:  1.0409589647247206
Error:  1.0404242374728103
Error:  1.0398891951032456
Error:  1.0393538381407987
Error:  1.0388181671126526
Error:  1.0382821825484054
Error:  1.037745884980066
Error:  1.0372092749420603
Error:  1.0366723529712258
Error:  1.0361351196068163
Error:  1.0355975753904976
Error:  1.0350597208663537
Error:  1.0345215565808807
Error:  1.0339830830829917
Error:  1.033444300924014
Error:  1.0329052106576904
Error:  1.0323658128401783
Error:  1.031826108030051
Error:  1.0312860967882977
Error:  1.0307457796783206
Error:  1.030205157265938
Error:  1.0296642301193835
Error:  1.0291229988093042
Error:  1.0285814639087618
Error:  1.0280396259932318
Error:  1.0274974856406038
Error:  1.0269550434311803
Error: 

...

Error:  0.21159189342453236
Error:  0.2115609821411058
Error:  0.21153011906157304
Error:  0.21149930411400353
Error:  0.21146853722657824
Error:  0.2114378183275855
Error:  0.21140714734542357
Error:  0.21137652420859976
Error:  0.21134594884572913
Error:  0.21131542118553656
Error:  0.21128494115685503
Error:  0.21125450868862475
Error:  0.2112241237098949
Error:  0.21119378614982304
Error:  0.21116349593767328
Error:  0.21113325300281804
Error:  0.21110305727473733
Error:  0.2110729086830176
Error:  0.21104280715735307
Error:  0.21101275262754451
Error:  0.21098274502350028
Error:  0.21095278427523406
Error:  0.21092287031286663
Error:  0.2108930030666255
Error:  0.21086318246684338
Error:  0.2108334084439602
Error:  0.2108036809285202
Error:  0.21077399985117393
Error:  0.2107443651426776
Error:  0.21071477673389255
Error:  0.2106852345557855
Error:  0.21065573853942765
Error:  0.2106262886159961
Error:  0.21059688471677077
Error:  0.2105675267731378
Error:  0.21053821471658712
Err

Error:  0.20190412043856515
Error:  0.20188858545796465
Error:  0.20187307588032755
Error:  0.20185759166829143
Error:  0.20184213278454918
Error:  0.2018266991918512
Error:  0.20181129085300248
Error:  0.20179590773086556
Error:  0.20178054978835733
Error:  0.2017652169884508
Error:  0.20174990929417533
Error:  0.2017346266686154
Error:  0.20171936907491056
Error:  0.20170413647625657
Error:  0.2016889288359038
Error:  0.20167374611715866
Error:  0.20165858828338198
Error:  0.20164345529798966
Error:  0.20162834712445338
Error:  0.20161326372629895
Error:  0.20159820506710688
Error:  0.2015831711105134
Error:  0.20156816182020829
Error:  0.20155317715993679
Error:  0.20153821709349767
Error:  0.20152328158474492
Error:  0.20150837059758678
Error:  0.20149348409598497
Error:  0.20147862204395572
Error:  0.20146378440556995
Error:  0.20144897114495178
Error:  0.20143418222627923
Error:  0.2014194176137846
Error:  0.20140467727175362
Error:  0.2013899611645256
Error:  0.2013752692564935


Error:  0.19775912028931084
Error:  0.19775058992662098
Error:  0.19774207471440244
Error:  0.19773357463059157
Error:  0.19772508965315802
Error:  0.19771661976010296
Error:  0.19770816492946122
Error:  0.19769972513929895
Error:  0.19769130036771587
Error:  0.19768289059284305
Error:  0.19767449579284438
Error:  0.1976661159459158
Error:  0.19765775103028585
Error:  0.19764940102421474
Error:  0.19764106590599503
Error:  0.19763274565395117
Error:  0.19762444024644008
Error:  0.19761614966185004
Error:  0.19760787387860165
Error:  0.19759961287514763
Error:  0.19759136662997134
Error:  0.19758313512158934
Error:  0.19757491832854937
Error:  0.1975667162294305
Error:  0.1975585288028437
Error:  0.19755035602743157
Error:  0.1975421978818683
Error:  0.19753405434485907
Error:  0.19752592539514138
Error:  0.1975178110114832
Error:  0.19750971117268448
Error:  0.19750162585757614
Error:  0.19749355504502009
Error:  0.19748549871391013
Error:  0.19747745684317097
Error:  0.197469429411758

Error:  0.19567288937219554
Error:  0.19566828550061172
Error:  0.19566369107578083
Error:  0.19565910608400863
Error:  0.1956545305116199
Error:  0.19564996434496018
Error:  0.19564540757039436
Error:  0.1956408601743074
Error:  0.19563632214310425
Error:  0.19563179346320939
Error:  0.19562727412106684
Error:  0.19562276410314075
Error:  0.19561826339591493
Error:  0.19561377198589266
Error:  0.19560928985959689
Error:  0.19560481700357046
Error:  0.195600353404375
Error:  0.19559589904859284
Error:  0.19559145392282506
Error:  0.19558701801369224
Error:  0.19558259130783473
Error:  0.19557817379191222
Error:  0.1955737654526035
Error:  0.1955693662766074
Error:  0.19556497625064126
Error:  0.1955605953614423
Error:  0.1955562235957669
Error:  0.19555186094039073
Error:  0.19554750738210844
Error:  0.19554316290773402
Error:  0.19553882750410118
Error:  0.19553450115806195
Error:  0.19553018385648754
Error:  0.195525875586269
Error:  0.19552157633431574
Error:  0.1955172860875561
Err

Error:  0.19457503789532643
Error:  0.19457296558231144
Error:  0.19457089904609104
Error:  0.19456883827828023
Error:  0.19456678327050722
Error:  0.19456473401441052
Error:  0.19456269050164274
Error:  0.19456065272386625
Error:  0.19455862067275703
Error:  0.19455659434000264
Error:  0.1945545737173022
Error:  0.19455255879636732
Error:  0.19455054956892087
Error:  0.1945485460266981
Error:  0.1945465481614465
Error:  0.19454455596492415
Error:  0.19454256942890275
Error:  0.19454058854516393
Error:  0.19453861330550246
Error:  0.19453664370172438
Error:  0.194534679725648
Error:  0.19453272136910232
Error:  0.19453076862392937
Error:  0.19452882148198206
Error:  0.19452687993512535
Error:  0.19452494397523595
Error:  0.19452301359420143
Error:  0.1945210887839226
Error:  0.19451916953631007
Error:  0.1945172558432876
Error:  0.19451534769678983
Error:  0.19451344508876328
Error:  0.19451154801116516
Error:  0.1945096564559653
Error:  0.19450777041514494
Error:  0.1945058898806967
E

## Comparison

In [13]:
print("True  -  Predicted")
tot_error = 0
for i in range(len(out)):
    print(f"{y_train[i]:.4f} - {out[i]:.4f}, Error: {y_train[i] - out[i]}")
    tot_error += (y_train[i] - out[i])**2
    
print("SSE: ",tot_error)

True  -  Predicted
0.0000 - 0.2086, Error: -0.20864836793335584
0.3311 - 0.3166, Error: 0.014576798047971129
0.4887 - 0.4770, Error: 0.01164306630919465
0.5429 - 0.6516, Error: -0.10878587095573311
0.5430 - 0.7840, Error: -0.24102125921008954
0.3461 - 0.3315, Error: 0.014588152138947486
0.6038 - 0.4962, Error: 0.10752990733934337
0.7260 - 0.6687, Error: 0.05729778973662947
0.7684 - 0.7951, Error: -0.02670421222549091
0.7676 - 0.8678, Error: -0.10018558098890429
0.5589 - 0.5162, Error: 0.04272143866098099
0.7715 - 0.6855, Error: 0.0859507865276189
0.8705 - 0.8055, Error: 0.06505980222775909
0.9036 - 0.8734, Error: 0.030199779778352864
0.9046 - 0.9091, Error: -0.004504722884265067
0.6767 - 0.7018, Error: -0.02505799001008102
0.8616 - 0.8152, Error: 0.046357999405587824
0.9519 - 0.8786, Error: 0.07335603769189203
0.9805 - 0.9118, Error: 0.06864649584831006
0.9778 - 0.9295, Error: 0.04823662455349509
0.7090 - 0.8244, Error: -0.11541967226822158
0.8877 - 0.8834, Error: 0.0043631137169207745

## 1.1 - Evaluate the result

In [14]:
# Start the evaluation
now = datetime.datetime.now()
result = []
tot = []

for s1, s2 in zip([2, 3, 1], [5, 9, 9]):
        
    p12, p23 = compute_probabilities(s1, s2)
    
    result.append(s1)
    result.append(s2)
    tot.append(s1)
    tot.append(s2)
    
    i_result = []
    tot_g = []
    for i in range(10000):

        tot_gain, last_gain = scheduling(p12, p23, s1, s2)
        i_result.append(last_gain)
        tot_g.append(tot_gain)
    result.append(i_result)
    tot.append(tot_g)

finish = datetime.datetime.now()

delta = finish - now

print(f"Start: {now}\nFinish: {finish}\nDeltaT:{delta}" )

Start: 2021-02-11 11:01:15.135432
Finish: 2021-02-11 11:01:24.255422
DeltaT:0:00:09.119990


In [15]:
# compute the expected values
expected_values = {}
array = []

final = []
for r in (tot[2::3]):
    #expected_values[(s1,s2)] = sum(r)/10000
    array.append(sum(r)/10000)

## Print result
i = 0
for s1, s2 in zip([0.2, 0.3, 0.1], [0.5, 0.9, 0.9]):
    out= net.think([s1,s2])
    gain = convert_to_gain(out)
    
    print(f"Expected gain from {s1}-{s2}: {array[i]:.2f} | Predicted: {gain:.2f}| AbsolutError: {abs(array[i]- gain)} ")
    i = i+1

Expected gain from 0.2-0.5: 94921.77 | Predicted: 94202.10| AbsolutError: 719.6676504862262 
Expected gain from 0.3-0.9: 99904.25 | Predicted: 102284.37| AbsolutError: 2380.1143464664783 
Expected gain from 0.1-0.9: 93874.53 | Predicted: 99302.51| AbsolutError: 5427.984560515804 


In [16]:
# Example of prediction: given an input it get the prediction normalized. Then denormalized.
result = net.think([0.5,0.9])
print("Result normalized:", result)
print("Gain:",convert_to_gain(result))

Result normalized: 0.8707513681747557
Gain: 103925.67786638328


## 1.2 - Optimization

In [17]:
# Box projection: input constraints [0, 1]
def box_projection(x):
    mins = [0,0]
    maxs = [1,1]
    xhat = np.maximum(np.minimum(x, maxs), mins);
    return xhat

In [24]:
# MU and C for Spall method
def mu(m):
    
    A = 500
    B = 1000
    l = 0.6
    return A/((B+m)**l)

def c(m):
    C=0.1
    t = 0.5
    return C/(m)**t

def spall_gradf(x, c):
    gradf = np.zeros(2)
    h = (2*np.random.randint(0,2,size=(2))-1) # h: random vector -1, 1
    x_plus = x.copy() + c* h
    x_minus = x.copy() - c* h
    
    gradf = (net.think(x_minus.tolist())- net.think(x_plus.tolist()))/(2*h)

    
    return gradf

In [19]:
# Armijo function
def farmijo(fk, alpha, xk, pk, h):
    farm = fk - c1* alpha*np.dot(gradf(xk, h),pk)
    return farm

In [25]:
# Finite difference
def gradf(x, h):
    gradf = np.zeros(2)
   
    # compute gradf
    for i in range(2):
        
        # centered finite difference
        x_plus = x.copy()
        x_minus = x.copy()
        x_plus[i]+= h
        x_minus[i]-=h
        #gradf[i] = (net.think(x_plus.tolist())- net.think(x_minus.tolist()))/(2*h)
        gradf[i] = (net.think(x_minus.tolist())- net.think(x_plus.tolist()))/(2*h)
    
    return gradf

In [26]:
x0 = np.random.rand(2) # random starting point between 0 and 1
print("Starting point:", x0)
print(f"Gain at x0: {convert_to_gain(net.think(x0))}")

# Init

h = 10**(-2)*np.linalg.norm(x0) # h value for finite difference
gamma = 0.01 #  factor that multiplies the descent direction before the (possible) projection
tolx = 10e-6 # a real scalar value characterizing the tolerance with respect to the 
               #norm of  |xk+1 - xk| in order to stop the method;
alpha0 = 0.5 #the initial factor that multiplies the descent direction at each
             #iteration;
c1 = 1e-4 # the factor of the Armijo condition that must be a scalar in (0,1);
rho = 0.5 # fixed factor, lesser than 1, used for reducing alpha0;
btmax = 50 # maximum number of steps for updating alpha during the backtracking strategy. 
tollgrad = 1e-12 #value used as stopping criterion w.r.t. the norm of the gradient;

x = x0.copy()
xk = box_projection(x) # projection 
kmax = 10000 # max number of iterations
k = 0 # init k for iterations

fk = net.think(xk)

gradfk_norm = np.linalg.norm(gradf(xk, h))
deltaxk_norm = tolx +1

spall = True # set True: use Spall Method, False: finite difference
while k<kmax and gradfk_norm >= tollgrad and deltaxk_norm >= tolx:


    if spall == True:
        # gradient
        pk = -spall_gradf(xk, c(k+1))
        gamma = mu(k+1)
        #update x and step-length
        xbark = xk + gamma*pk
        
        # box proj. if exceed the constraint
        xhatk = box_projection(xbark)
        
        xnew = xhatk
        fnew = net.think(xnew)
        deltaxk_norm = np.linalg.norm(xnew - xk)
        xk = xnew
        fk = fnew
        gradfk_norm = np.linalg.norm(spall_gradf(xk, c(k+1)));
    else:
        # gradient
        pk = -gradf(xk, h)
        #update x and step-length
        xbark = xk + gamma*pk

        # box proj. if exceed the constraints

        xhatk = box_projection(xbark)
    
        
        # Reset the value of alpha for Armijo
        alpha = alpha0

        #compute the candidate xk
        pik = xhatk - xk

        xnew = xk + alpha*pik

        # compute the function f in xnew
        fnew = net.think(xnew)
        
        #Backtracing strategy with Armijo condition
        bt = 0
    
        while bt<btmax and fnew < farmijo(fk,alpha, xk, pik,h):
            #Reduce the value of alpha
            alpha = rho * alpha
            # Update xnew and fnew w.r.t. the reduced alpha
            xnew = xk + alpha * pik
            fnew = net.think(xnew)
            

            # increase the counter
            bt = bt + 1


        # Update xk, fk, gradfk_norm, deltaxk_norm
        deltaxk_norm = np.linalg.norm(xnew - xk)
        
        xk = xnew
        fk = fnew
        gradfk_norm = np.linalg.norm(gradf(xk, h));
      
    
    # Increase the step by one
    k = k + 1;
       

print(f"-------------------------------------\nEnd point at iteration K:{k+1} ,xk:{xk}")
if spall == True:
    print("Spall method")
else:
    print("Finite difference")

res = net.think(xk)
print(f"Result:{fnew}")

print(f"Optimized gain:{convert_to_gain(fnew)}")


Starting point: [0.18388072 0.40347368]
Gain at x0: 92487.08629955041
-------------------------------------
End point at iteration K:26 ,xk:[1. 1.]
Spall method
Result:0.9401809610573094
Optimized gain:105334.49653624148
